In [1]:
# Create normalized dataset
# - duplicates https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/52752
# - join test_supplement.csv and test.csv
# - sort by click_time

In [2]:
import pandas as pd
import numpy as np

In [20]:
DATA_DIR = '../data/mnt/ssd/kaggle-talkingdata2/competition_files'
TRAIN_SAMPLE_FILE = DATA_DIR + '/train_sample.csv'
TRAIN_FILE = DATA_DIR + '/train.csv'
TEST_FILE = DATA_DIR + '/test.csv'
TEST_SUPPLEMENT_FILE = DATA_DIR + '/test_supplement.csv'
TEST_JOINED_FILE = DATA_DIR + '/test_joined.csv'

In [4]:
# test_supplement.csv
df_test_supplement = pd.read_csv(TEST_SUPPLEMENT_FILE)

In [5]:
# test.csv: this is a subset of test_supplement.csv which is used to score submissions
df_test = pd.read_csv(TEST_FILE)

In [6]:
# join test_supplement.csv and test.csv
df_test_joined = df_test_supplement.merge(df_test, how='left', on=['ip', 'app', 'device', 'os', 'channel', 'click_time'], suffixes=['', '_submission'])

In [7]:
# remove extra duplicates
# note: pandas consider duplicates rows that are identical even if there are nans in some columns
duplicated_idx = df_test_joined.duplicated(subset=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id_submission'], keep='first')
df_test_joined_dedup = df_test_joined[(~ duplicated_idx) | (df_test_joined['click_id_submission'].isnull())]

In [8]:
# check that all subsmission clicks are preserved after join and remove of extra duplicates
df_test_joined_dedup['click_id_submission'].value_counts().sum()
assert df_test.shape[0] == df_test_joined_dedup['click_id_submission'].value_counts().shape[0]
assert df_test.shape[0] == df_test_joined_dedup['click_id_submission'].value_counts().sum()

In [9]:
# note: a small number of events from test_supplement is lost after deduplications. 
# Assuming these were events in test_supplement.csv which were not present in test.csv but still were duplicates of events from test.csv
print(df_test_supplement.shape[0] - df_test_joined_dedup.shape[0])

33


In [10]:
del(df_test_supplement)
del(df_test)
del(df_test_joined)

In [11]:
df_test_joined_dedup.sort_values(by=['click_time', 'ip', 'app', 'device', 'os', 'channel'], inplace=True)

In [16]:
df_test_joined_dedup['click_id_submission'] = df_test_joined_dedup['click_id_submission'].fillna(value=-1).astype(int)

In [21]:
df_test_joined_dedup.to_csv(TEST_JOINED_FILE, index=False)